In [ ]:
def station32_34_tr():
    
    STATIONS = ['S32', 'S33', 'S34']
    train_date_part = pd.read_csv('Y://Data//Kaggle//bosch//train_date//train_date.csv')
    date_cols = train_date_part.drop('Id', axis=1).count().reset_index().sort_values(by=0, ascending=False)
    date_cols['station'] = date_cols['index'].apply(lambda s: s.split('_')[1])
    date_cols = date_cols[date_cols['station'].isin(STATIONS)]
    date_cols = date_cols.drop_duplicates('station', keep='first')['index'].tolist()
    print(date_cols)
    train_date = pd.read_csv('Y://Data//Kaggle//bosch//train_date//train_date.csv', usecols=['Id'] + date_cols)
    print(train_date.columns)
    train_date.columns = ['Id'] + STATIONS
    for station in STATIONS:
        train_date[station] = 1 * (train_date[station] >= 0)
    response = pd.read_csv('Y://Data//Kaggle//bosch//train_numeric.csv', usecols=['Id'])
    print(response.shape)
    train = response.merge(train_date, how='left', on='Id')
    # print(train.count())
    #train.head(3)
    return train

In [ ]:
def station32_34_te():
    
    STATIONS = ['S32', 'S33', 'S34']
    test_date_part = pd.read_csv('Y://Data//Kaggle//bosch//test_date.csv')
    date_cols = test_date_part.drop('Id', axis=1).count().reset_index().sort_values(by=0, ascending=False)
    date_cols['station'] = date_cols['index'].apply(lambda s: s.split('_')[1])
    date_cols = date_cols[date_cols['station'].isin(STATIONS)]
    date_cols = date_cols.drop_duplicates('station', keep='first')['index'].tolist()
    print(date_cols)
    test_date = pd.read_csv('Y://Data//Kaggle//bosch//test_date.csv', usecols=['Id'] + date_cols)
    print(test_date.columns)
    test_date.columns = ['Id'] + STATIONS
    for station in STATIONS:
        test_date[station] = 1 * (test_date[station] >= 0)
    response = pd.read_csv('Y://Data//Kaggle//bosch//test_numeric.csv', usecols=['Id'])
    print(response.shape)
    test = response.merge(test_date, how='left', on='Id')

    return test

In [ ]:
tr=station32_34_tr()
tr.to_csv('Y://Data//Kaggle//bosch//station_feature_tr.csv',index=False)
te=station32_34_te()
te.to_csv('Y://Data//Kaggle//bosch//station_feature_te.csv',index=False)

In [ ]:
#very inspiring features, start time & end time etc.,  
def magic_feature():

    DATA_DIR = "Y://Data//Kaggle//bosch"

    ID_COLUMN = 'Id'
    TARGET_COLUMN = 'Response'

    SEED = 0
    CHUNKSIZE = 51500
    NROWS_TR = 1183747
    NROWS_TE = 1183748

    TRAIN_NUMERIC = "{0}/train_numeric.csv".format(DATA_DIR)
    TRAIN_DATE = "{0}/train_date.csv".format(DATA_DIR)


    TEST_NUMERIC = "{0}/test_numeric.csv".format(DATA_DIR)
    TEST_DATE = "{0}/test_date.csv".format(DATA_DIR)


    FILENAME = "etimelhoods"

    train = pd.read_csv(TRAIN_NUMERIC, usecols=[ID_COLUMN, TARGET_COLUMN], nrows=NROWS_TR)
    test = pd.read_csv(TEST_NUMERIC, usecols=[ID_COLUMN], nrows=NROWS_TE)

    train["StartTime"] = -1
    test["StartTime"] = -1
    train["EndTime"]= -1
    test["EndTime"]= -1
    train["Duration"]=-1
    test["Duration"]=-1


    #date process
    #nrows = 0
    for tr in pd.read_csv(TRAIN_DATE, chunksize=CHUNKSIZE):

        feats = np.setdiff1d(tr.columns, [ID_COLUMN])
        stime_tr = tr[feats].min(axis=1).values #min value of col
        etime_tr = tr[feats].max(axis=1).values
        du_tr=etime_tr - stime_tr


        train.loc[train.Id.isin(tr.Id), 'StartTime'] = stime_tr
        train.loc[train.Id.isin(tr.Id), 'EndTime'] = etime_tr
        train.loc[train.Id.isin(tr.Id), 'Duration'] = du_tr



        #nrows += CHUNKSIZE

        #if nrows >= NROWS:
            #break 
            
    for  te in pd.read_csv(TEST_DATE, chunksize=CHUNKSIZE):

        feats = np.setdiff1d(tr.columns, [ID_COLUMN])

        stime_te = te[feats].min(axis=1).values
        etime_te = te[feats].max(axis=1).values
        du_te=etime_te - stime_te

        test.loc[test.Id.isin(te.Id), 'StartTime'] = stime_te
        test.loc[test.Id.isin(te.Id), 'EndTime'] = etime_te
        test.loc[test.Id.isin(te.Id), 'Duration'] = du_te


#        nrows += CHUNKSIZE
#        if nrows >= NROWS:
#            break

    ntrain = train.shape[0]
    train_test = pd.concat((train, test)).reset_index(drop=True).reset_index(drop=False)

    train_test['m1'] = train_test[ID_COLUMN].diff().fillna(9999999).astype(int)
    train_test['m2'] = train_test[ID_COLUMN].iloc[::-1].diff().fillna(9999999).astype(int)

    train_test = train_test.sort_values(by=['StartTime', 'Id'], ascending=True)
    
    train_test['m3'] = train_test[ID_COLUMN].diff().fillna(9999999).astype(int)
    train_test['m4'] = train_test[ID_COLUMN].iloc[::-1].diff().fillna(9999999).astype(int)

    train_test = train_test.sort_values(by=['index']).drop(['index'], axis=1)

    train = train_test.iloc[:ntrain, :]
    
    test  = train_test.iloc[ntrain:, :]

    features = np.setdiff1d(list(train.columns), [TARGET_COLUMN])
    train=train[features]
    
    test=test[features]
    
    return train, test


In [ ]:
#write to save time cost
tr,te= magic_feature()
tr.to_csv('Y://Data//Kaggle//bosch//magic_feature_tr_1104.csv',index=False)
te.to_csv('Y://Data//Kaggle//bosch//magic_feature_te_1104.csv',index=False)

In [ ]:
def uniq_ctg(s):
    n=len(s.unique())
    return n

def ctg_self_feature():
    
    DATA_DIR = "Y://Data//Kaggle//bosch"

    ID_COLUMN = 'Id'
    TARGET_COLUMN = 'Response'

    SEED = 0
    CHUNKSIZE = 51500
    NROWS_TR = 1183747
    NROWS_TE = 1183748

    TRAIN_NUMERIC = "{0}/train_numeric.csv".format(DATA_DIR)
    TRAIN_CTG = "{0}/train_categorical.csv".format(DATA_DIR)
    TEST_NUMERIC = "{0}/test_numeric.csv".format(DATA_DIR)


    TEST_CTG = "{0}/test_categorical.csv".format(DATA_DIR)


    FILENAME = "etimelhoods"

    train = pd.read_csv(TRAIN_NUMERIC, usecols=[ID_COLUMN], nrows=NROWS_TR)
    test = pd.read_csv(TEST_NUMERIC, usecols=[ID_COLUMN], nrows=NROWS_TE)

    train["No_NaN"] = -1
    train["uniqv"]= -1
    test["No_NaN"] = -1
    test["uniqv"]= -1

    for tr in pd.read_csv(TRAIN_CTG, chunksize=CHUNKSIZE, low_memory=False):

        #feats = np.setdiff1d(tr.columns, [ID_COLUMN])
        nonan= tr.count(axis=1).tolist()
        #uniq=tr.apply(pd.Series.nunique,axis=1).tolist()
        uniq= tr.apply(uniq_ctg, axis=1).tolist()

        train.loc[train.Id.isin(tr.Id), 'No_NaN'] = nonan
        
        train.loc[train.Id.isin(tr.Id), 'uniqv'] = uniq
        print(1)
        

            
    for  te in pd.read_csv(TEST_CTG, chunksize=CHUNKSIZE,low_memory=False):

        #feats = np.setdiff1d(tr.columns, [ID_COLUMN])

        nonan1 = te.count(axis=1).tolist()
        #etime_te = te[feats].max(axis=1).values
        
        uniq= te.apply(uniq_ctg, axis=1).tolist()

        test.loc[test.Id.isin(te.Id), 'No_NaN'] = nonan1
        test.loc[test.Id.isin(te.Id), 'uniqv'] = uniq
        
        print(2)

 
    
    return train, test